In [ ]:
# Importar bibliotecas necessárias
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [ ]:
# Baixar e extrair o dataset
!wget --no-check-certificate \
    https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip \
    -O /tmp/cats-and-dogs.zip

import zipfile
local_zip = '/tmp/cats-and-dogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
# Definir diretórios
base_dir = '/tmp/PetImages'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Criar diretórios de treino e validação
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

# Criar subdiretórios para cada classe
train_cats_dir = os.path.join(train_dir, 'Cat')
train_dogs_dir = os.path.join(train_dir, 'Dog')
validation_cats_dir = os.path.join(validation_dir, 'Cat')
validation_dogs_dir = os.path.join(validation_dir, 'Dog')

os.makedirs(train_cats_dir, exist_ok=True)
os.makedirs(train_dogs_dir, exist_ok=True)
os.makedirs(validation_cats_dir, exist_ok=True)
os.makedirs(validation_dogs_dir, exist_ok=True)

In [ ]:
# Função para dividir os dados
def split_data(source, train, validation, split_size):
    files = []
    for filename in os.listdir(source):
        file = os.path.join(source, filename)
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(f"{filename} está vazio, ignorando.")

    train_files, val_files = train_test_split(files, test_size=split_size, random_state=42)

    for filename in train_files:
        src = os.path.join(source, filename)
        dst = os.path.join(train, filename)
        os.rename(src, dst)

    for filename in val_files:
        src = os.path.join(source, filename)
        dst = os.path.join(validation, filename)
        os.rename(src, dst)

# Aplicar a divisão
split_size = 0.2
split_data(os.path.join(base_dir, 'Cat'), train_cats_dir, validation_cats_dir, split_size)
split_data(os.path.join(base_dir, 'Dog'), train_dogs_dir, validation_dogs_dir, split_size)

In [ ]:
# Configurar geradores de dados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

# Geradores de fluxo de dados
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

In [ ]:
# Carregar modelo base VGG16 (pré-treinado no ImageNet)
base_model = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

# Congelar camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False

# Criar modelo completo
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compilar o modelo
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

# Resumo do modelo
model.summary()

In [ ]:
# Treinar o modelo
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50)

In [ ]:
# Plotar acurácia e loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_animal(image_path, model):
    # Carregar a imagem e redimensionar para o tamanho esperado pelo modelo (150x150)
    img = image.load_img(image_path, target_size=(150, 150))

    # Converter a imagem para um array numpy
    img_array = image.img_to_array(img)

    # Adicionar uma dimensão extra (porque o modelo espera um batch de imagens)
    img_array = np.expand_dims(img_array, axis=0)

    # Normalizar os valores dos pixels (igual fizemos no treinamento)
    img_array /= 255.0

    # Fazer a predição
    prediction = model.predict(img_array)

    # Interpretar o resultado
    if prediction[0] < 0.5:
        return "Gato", 1 - prediction[0][0]
    else:
        return "Cachorro", prediction[0][0]

# Função para exibir a imagem e a predição
def show_prediction(image_path, model):
    class_name, confidence = predict_animal(image_path, model)

    img = image.load_img(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Predição: {class_name} ({confidence*100:.2f}% de confiança)")
    plt.show()

In [ ]:
# Exemplo com uma imagem de cachorro
!wget https://images.unsplash.com/photo-1561037404-61cd46aa615b -O dog_test.jpg

show_prediction("dog_test.jpg", model)

# Exemplo com uma imagem de gato
!wget https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba -O cat_test.jpg

show_prediction("cat_test.jpg", model)